# Slack client

In [1]:
import requests
from bs4 import BeautifulSoup as soup
import os.path
import os
import json
from datetime import datetime
from datetime import timedelta
import ast


# https://api.slack.com/apps/APSE1SZPZ/incoming-webhooks?
class SlackNotifier():
    def __init__(self, url, interval_in_s):
        self.slack_hook_url = url
        self.tdelta = timedelta(seconds=interval_in_s)
        self.lasttime = {}
    def call(self, Id, message, forced=False):
        if forced:
            requests.post(self.slack_hook_url, json={"text":message})
            self.lasttime[Id] = datetime.now()
        elif not self.is_just_called(Id):
            requests.post(self.slack_hook_url, json={"text":message})
            self.lasttime[Id] = datetime.now()
    def is_just_called(self, Id):
        if not self.lasttime:
            return False
        elif Id in self.lasttime.keys():
            return datetime.now() < self.lasttime[Id] + self.tdelta        
        else:
            return False

In [2]:
notifier = SlackNotifier(os.environ['SLACK_HOOK_URL'], 10)

In [3]:
notifier.call(0, "FOR KVR RESIDENCE PERMIT: NEW MESSAGE.")

# Check available dates

In [4]:
# For other termines, e.g An- oder Ummeldung - Einzelperson
TERMIN_URL = "https://www56.muenchen.de/termin/index.php?loc=BB"
CASETYPES = 'CASETYPES[An- oder Ummeldung - Einzelperson]'
ZONE = 'Termin Wartezone 1 P'

# For SCIF
# TERMIN_URL = "https://www46.muenchen.de/termin/index.php?cts=1080627"
# CASETYPES = 'CASETYPES[Aufenthaltserlaubnis Blaue Karte EU]'
# ZONE = 'Termin Wartezone SCIF'

# For Fuehrer
TERMIN_URL = "https://www22.muenchen.de/view-fs/termin/index.php?"
CASETYPES = 'CASETYPES[FS Fahrerlaubnis erstmalig]'
ZONE = 'Termin FS Umtauschstelle_G'

def check_KRV_SCIF_available_date(termin_url, casetypes, zone):
    # Get information for KVR from
    # https://www46.muenchen.de/termin/index.php?cts=1080627"
    termin_url = termin_url
    
    # First call to get cookies and __ncforminfo
    headers1 = {
        'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1',
        'Connection' : 'keep-alive',
        'Cache-Control' : 'max-age=0',
        'Upgrade-Insecure-Requests' : '1',
        'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding' : 'gzip, deflate, br',
        'Accept-Language' : 'en-US,en;q=0.9,de;q=0.8,fr;q=0.7,vi;q=0.6',
        'Cookie' : '_et_coid=e32c2d08fe2aec5f79ee9875c9f20463'
    }
    r1 = requests.get(termin_url, headers=headers1)
    page = r1.content
    doc = soup(page, "html.parser")
#     element = doc.findAll('input', {"name": "__ncforminfo"})[0]
    
    # Second call to get available dates
    data = {}
    data['step'] = 'WEB_APPOINT_SEARCH_BY_CASETYPES'
    data[casetypes] = '1'
#     data['__ncforminfo'] = element['value']

    headers2 = {
        'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1',
        'Connection' : 'keep-alive',
        'Cache-Control' : 'max-age=0',
        'Upgrade-Insecure-Requests' : '1',
        'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding' : 'gzip, deflate, br',
        'Accept-Language' : 'en-US,en;q=0.9,de;q=0.8,fr;q=0.7,vi;q=0.6',
    }

    r2 = requests.post(termin_url, cookies=r1.cookies, headers=headers2, data=data)
    page = r2.content
    page = page.decode("utf-8")

    for item in page.split("\n"):
        if "jsonAppoints" in item:
            appoints_line = item.strip()
            break
    right = appoints_line.find('{')
    left = appoints_line.rfind('}')
    appoints = ast.literal_eval(appoints_line[right:left+1])
    
    appoints_dates = appoints[zone]['appoints']
    available_date = None
    for key in appoints_dates.keys():
        if len(appoints_dates[key]) > 0:
            available_date = key
            break
    
    return available_date

In [5]:
a = check_KRV_SCIF_available_date(TERMIN_URL, CASETYPES, ZONE)

In [6]:
a

'2020-05-13'

In [7]:
str(None)

'None'

In [7]:
print(check_KRV_SCIF_available_date(TERMIN_URL, CASETYPES, ZONE))

2019-11-11


# Delete process script

In [54]:
import os

if os.path.exists("/tmp/terminvereinbarung.pid"):
    with open("/tmp/terminvereinbarung.pid", "r") as f:
        current_pid = f.readline()
        current_pid = current_pid.split("\n")[0]
        kill_syntax = "kill -9 {}".format(current_pid)
        os.system(kill_syntax)
        if os.path.exists("/tmp/terminvereinbarung.pid.lock"):
            os.remove("/tmp/terminvereinbarung.pid.lock")

In [ ]:
datetime.